In [ ]:
from pyspark.streaming import StreamingContext
batchIntervalSeconds = 10 

ssc = StreamingContext(spark.sparkContext, batchIntervalSeconds)
# Set each DStreams in this context to remember RDDs it generated in the last given duration.
# DStreams remember RDDs only for a limited duration of time and releases them for garbage
# collection. This method allows the developer to specify how long to remember the RDDs (
# if the developer wishes to query old data outside the DStream computation).
ssc.remember(60)
lines = ssc.textFileStream("s3://sita-coe-ds-flightaware/")

ssc.sparkSession(lines.pprint()
words = lines.flatMap(lambda line: line.split(","))
pairs = words.map(lambda word: (word, 1))
wordCounts = pairs.reduceByKey(lambda x, y: x + y)

def process(time, rdd):
    df = sqlContext.createDataFrame(rdd)
    df.registerTempTable("myCounts")
    df.show()


wordCounts.foreachRDD(process)
wordCounts.pprint()

checkpointDir = "/mnt/tmp/checkpoint"

# This line starts the streaming context in the background.
ssc.start()

# This ensures the cell is put on hold until the background streaming thread is started properly.
ssc.awaitTerminationOrTimeout(batchIntervalSeconds * 2)
#ssc.stop()


In [ ]:
#from __future__ import print_function

#import sys

#from pyspark import SparkContext
from pyspark.streaming import StreamingContext
batchIntervalSeconds = 10 

def creatingFunc():
  ssc = StreamingContext(spark.sparkContext, batchIntervalSeconds)
  # Set each DStreams in this context to remember RDDs it generated in the last given duration.
  # DStreams remember RDDs only for a limited duration of time and releases them for garbage
  # collection. This method allows the developer to specify how long to remember the RDDs (
  # if the developer wishes to query old data outside the DStream computation).
  ssc.remember(60)
  lines = ssc.textFileStream("s3://sita-coe-ds-pre-prod-v1/stream/flightaware/")
  lines.pprint()
  words = lines.flatMap(lambda line: line.split(","))
  pairs = words.map(lambda word: (word, 1))
  wordCounts = pairs.reduceByKey(lambda x, y: x + y)
  def process(time, rdd):
    df = sqlContext.createDataFrame(rdd)
    df.registerTempTable("myCounts")
  wordCounts.foreachRDD(process)
  return ssc


checkpointDir = "/mnt/tmp/checkpoint"
ssc = StreamingContext.getActiveOrCreate(checkpointDir, creatingFunc)

# This line starts the streaming context in the background.
ssc.start()

# This ensures the cell is put on hold until the background streaming thread is started properly.
ssc.awaitTerminationOrTimeout(batchIntervalSeconds * 2)

sc = SparkContext(appName="PythonStreamingNetworkWordCount")
ssc = StreamingContext(spark.sparkContext, 1)

lines = ssc.textFileStream('s3://sita-coe-ds-pre-prod-v1/stream/flightaware/')
#counts = lines.flatMap(lambda line: line.split(" "))\
#              .map(lambda word: (word, 1))\
#              .reduceByKey(lambda a, b: a+b)

def sendRecord(rdd):
    connection = createNewConnection()  # executed at the driver
    rdd.foreach(lambda record: connection.send(record))
    connection.close()

#lines.foreachRDD(print)


ssc.start()
ssc.awaitTermination()


In [ ]:
counts
